In [1]:
from ngsolve import *
import netgen.geom2d as g2d
from netgen.meshing import Pnt

In [2]:
#from netgen.csg import *

#cube = OrthoBrick( Pnt(0,0,0), Pnt(1,1,1) )
#geo = CSGeometry()
#geo.Add (cube)
#ngmesh = geo.GenerateMesh(maxh=0.1)
#mesh = Mesh(ngmesh)

In [3]:
geo = g2d.CSG2d()
geo.Add(g2d.Circle(Pnt((0,0)), 1.0))
ngmesh = geo.GenerateMesh(maxh=1.0)
mesh = Mesh(ngmesh)
mesh.ngmesh.SecondOrder()
#mesh.Curve(2)
Draw(mesh)

In [4]:
V = VectorH1(mesh, order=2)
u,v = V.TnT()

ndscal = V.ndof//mesh.dim
nv = mesh.nv
ne = mesh.ne
nedges = len(mesh.edges)
print("nv:", nv)
print("ne:", ne)
print("nedges:", nedges)
print("ndof:",V.ndof)
print("ndscal:",ndscal)

nv: 20
ne: 26
nedges: 45
ndof: 130
ndscal: 65


In [5]:
#for i in range(V.ndof):
#    print(V.CouplingType(i))
#
#print("+++++++++++++++++++++")

In [6]:
V.SetCouplingType(IntRange(0, V.ndof), COUPLING_TYPE.LOCAL_DOF)
# for k,v in enumerate(mesh.vertices):
#     V.SetCouplingType(k, WIREBASKET_DOF)
#     V.SetCouplingType(ndscal+k, WIREBASKET_DOF)
V.SetCouplingType(IntRange(0, nv), COUPLING_TYPE.WIREBASKET_DOF)
V.SetCouplingType(IntRange(ndscal, ndscal+nv), COUPLING_TYPE.WIREBASKET_DOF)

In [7]:
#for i in range(V.ndof):
#    print(V.CouplingType(i))

In [8]:
eps = lambda U : Grad(U)+Grad(U).trans
a = BilinearForm(V, eliminate_internal=True)
a += InnerProduct(eps(u), eps(v))*dx
a.Assemble()

In [9]:
gfu = GridFunction(V)


# gfu.vec[:] = 1.0

ndscal = V.ndof//mesh.dim
nv = mesh.nv
for k,v in enumerate(mesh.vertices):
    x,y = v.point
    gfu.vec[k] = -y
    gfu.vec[ndscal+k] = x

In [10]:
print("\nA-norm**2 = {}\n".format(InnerProduct(a.mat*gfu.vec,gfu.vec)))
# print("\nc-norm**2 = {}\n".format(InnerProduct(c.mat*gfu.vec,gfu.vec)))
Draw(gfu)


A-norm**2 = 5.197856952839326e-16

